##  MLflow Model Registry Integration

### 1. `mlflow.set_tracking_uri()` (Optional)

```python
# mlflow.set_tracking_uri("http://localhost:5000")
```

#### What it does:

This line lets you specify a **remote or local MLflow tracking server**. If you're running MLflow on a different machine or in the cloud, you set the URI here. If commented out, the script uses the default local backend (e.g., `mlruns/` folder).

#### Use case:

Use this if you want to:

* Log runs to a central MLflow server
* Enable collaboration with teammates
* Store experiments persistently

---

### 2. `with mlflow.start_run() as run:`

This starts a run and stores the `run` object, which includes metadata such as `run_id`. You need this ID to register a model later.

```python
with mlflow.start_run() as run:
    ...
```

---

### 3. Registering the Model in the MLflow Registry

```python
model_uri = f"runs:/{run.info.run_id}/model"
mlflow.register_model(model_uri=model_uri, name="BankLogisticModel")
```

#### What it does:

This **registers the trained model** in MLflow’s **Model Registry** under the name `BankLogisticModel`.

* `runs:/<run_id>/model` is the path to the model saved in that specific experiment run.
* `name="BankLogisticModel"` creates a new entry in the registry (or a new version if it already exists).

#### Why it's useful:

MLflow Model Registry allows you to:

* Track model versions (e.g., version 1, 2, ...)
* Add stage labels like `Staging`, `Production`, `Archived`
* Promote/demote models in a CI/CD workflow
* Serve and deploy models reliably

In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Optional: Set remote tracking URI if needed
# mlflow.set_tracking_uri("http://localhost:5000")

# Use autologging
mlflow.sklearn.autolog()

# Load dataset
df = pd.read_csv("bankdata.csv")

# Encode categorical columns
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = LabelEncoder().fit_transform(df[col])

X = df.drop("y", axis=1)
y = df["y"]

# Scale the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model parameter
C = 0.1

# Start MLflow run
with mlflow.start_run() as run:
    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    acc = accuracy_score(y_test, model.predict(X_test))
    print(f"Accuracy: {acc:.4f}")

    # Register the model
    model_uri = f"runs:/{run.info.run_id}/model"
    mlflow.register_model(model_uri=model_uri, name="BankLogisticModel")